In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE as smt
from imblearn.under_sampling import RandomUnderSampler as rrs

from pycaret.classification import *

from glob import glob

import os
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(f'./cltandonehot_final.csv', sep=';')
data.columns

Index(['24_Hour_Peak', 'All_time_peak', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'num_lang',
       'required_age',
       ...
       'Linear', 'Grid-Based Movement', 'FMV', 'Lemmings', 'Tile-Matching',
       'Indie', 'Arcade', 'Atmospheric', 'Cats', 'tag_num'],
      dtype='object', length=511)

In [3]:
data.filter(regex=r'c\d+').head().columns

Index(['c0', 'c1', 'c2', 'c3', 'c4', 'c5'], dtype='object')

In [4]:
train, test = train_test_split(data, random_state=1234,
                            stratify=data['Review'])

In [5]:
print(train.shape, test.shape)

(34689, 511) (11563, 511)


In [6]:
caret = setup(train, target='Review', session_id=1234, fold=5)

,Description,Value
0,Session id,1234
1,Target,Review
2,Target type,Binary
3,Target mapping,"Negative: 0, Positive: 1"
4,Original data shape,"(34689, 511)"
5,Transformed data shape,"(34689, 511)"
6,Transformed train set shape,"(24282, 511)"
7,Transformed test set shape,"(10407, 511)"
8,Numeric features,510
9,Preprocess,True


In [7]:
comp = caret.compare_models(n_select=5, fold=5, sort='Recall')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:48:46
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


In [8]:
init_report = pull()
init_report

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7367,0.7992,0.6145,0.7178,0.6621,0.4486,0.4523,0.600
catboost,CatBoost Classifier,0.7429,0.8079,0.6097,0.7331,0.6656,0.4596,0.4648,0.824
lightgbm,Light Gradient Boosting Machine,0.7384,0.8046,0.6070,0.7253,0.6608,0.4506,0.4554,0.832
rf,Random Forest Classifier,0.7375,0.7983,0.5856,0.7352,0.6519,0.4455,0.4529,0.716
dt,Decision Tree Classifier,0.6483,0.6392,0.5824,0.5811,0.5817,0.2783,0.2783,0.526
gbc,Gradient Boosting Classifier,0.7287,0.7914,0.5586,0.7320,0.6336,0.4243,0.4342,0.576
ada,Ada Boost Classifier,0.7193,0.7768,0.5575,0.7118,0.6252,0.4061,0.4140,0.816
et,Extra Trees Classifier,0.7147,0.7671,0.5505,0.7053,0.6183,0.3960,0.4038,0.886
qda,Quadratic Discriminant Analysis,0.5291,0.5325,0.5087,0.4676,0.4105,0.0532,0.0584,0.658
lda,Linear Discriminant Analysis,0.6729,0.7221,0.5033,0.6406,0.5636,0.3080,0.3138,0.522


In [9]:
xgb = comp[0]
xgb.random_state

1234

In [10]:
# OVERSAMPLING

sm = smt(n_jobs=-1, random_state=1234)

r_train_x, r_train_y = sm.fit_resample(X =train.drop('Review', axis=1), y = train['Review'])

In [11]:
r_train = r_train_x.copy()
r_train['Review'] = r_train_y

In [12]:
print(train.shape, r_train.shape)

(34689, 511) (40250, 511)


In [13]:
over_set = setup(r_train, fold=5, target='Review', session_id=1234)
or_xgb = over_set.create_model('xgboost', fold=5, return_train_score=True, random_state=1234)

,Description,Value
0,Session id,1234
1,Target,Review
2,Target type,Binary
3,Target mapping,"Negative: 0, Positive: 1"
4,Original data shape,"(40250, 511)"
5,Transformed data shape,"(40250, 511)"
6,Transformed train set shape,"(28175, 511)"
7,Transformed test set shape,"(12075, 511)"
8,Numeric features,510
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:50:15
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
or_xgb_report = pull()

In [15]:
# UNDERSAMPLING

rs = rrs(random_state=1234)

rs_train_x, rs_train_y = rs.fit_resample(X =train.drop('Review', axis=1), y = train['Review'])

In [16]:
rs_train = rs_train_x.copy()
rs_train['Review'] = rs_train_y

In [17]:
print(rs_train.shape, train.shape)

(29128, 511) (34689, 511)


In [18]:
under_set = setup(rs_train, fold=5, target='Review', session_id=1234)
ur_xgb = under_set.create_model('xgboost', fold=5, return_train_score=True, random_state=1234)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:52:03
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
ur_xgb_report = pull()

In [20]:
data2 = data.drop(['c0', 'c1', 'c2', 'c3', 'c4', 'c5'],axis=1)

train2, test2 = train_test_split(data2, random_state=1234,
                            stratify=data2['Review'])

caret2 = setup(train2, target='Review', session_id=1234, fold=5)
comp2 = caret2.compare_models(n_select=5, fold=5, sort='Recall')
init_report2 = pull()
init_report2

,Description,Value
0,Session id,1234
1,Target,Review
2,Target type,Binary
3,Target mapping,"Negative: 0, Positive: 1"
4,Original data shape,"(34689, 505)"
5,Transformed data shape,"(34689, 505)"
6,Transformed train set shape,"(24282, 505)"
7,Transformed test set shape,"(10407, 505)"
8,Numeric features,504
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:53:27
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7377,0.8003,0.6124,0.7210,0.6621,0.4501,0.4542,12.288
catboost,CatBoost Classifier,0.7426,0.8075,0.6098,0.7326,0.6655,0.4592,0.4644,15.368
lightgbm,Light Gradient Boosting Machine,0.7394,0.8039,0.6092,0.7261,0.6624,0.4528,0.4575,1.062
rf,Random Forest Classifier,0.7384,0.7997,0.5830,0.7389,0.6517,0.4469,0.4550,3.482
dt,Decision Tree Classifier,0.6465,0.6375,0.5809,0.5790,0.5798,0.2747,0.2748,1.228
ada,Ada Boost Classifier,0.7191,0.7764,0.5575,0.7111,0.6249,0.4056,0.4133,2.864
gbc,Gradient Boosting Classifier,0.7279,0.7912,0.5570,0.7310,0.6322,0.4225,0.4324,6.372
et,Extra Trees Classifier,0.7122,0.7675,0.5481,0.7012,0.6153,0.3909,0.3985,5.160
knn,K Neighbors Classifier,0.6556,0.6728,0.4918,0.6120,0.5452,0.2734,0.2777,3.246
svm,SVM - Linear Kernel,0.4680,0.0000,0.4774,0.4761,0.4431,-0.0397,-0.0560,0.710


In [21]:
xgb2 = comp2[0]

In [22]:
# OVERSAMPLING

sm = smt(n_jobs=-1, random_state=1234)

r_train_x2, r_train_y2 = sm.fit_resample(X =train2.drop('Review', axis=1), y = train2['Review'])

r_train2 = r_train_x2.copy()
r_train2['Review'] = r_train_y2

over_set2 = setup(r_train2, fold=5, target='Review', session_id=1234)
or_xgb2 = over_set2.create_model('xgboost', fold=5, return_train_score=True, random_state=1234)

,Description,Value
0,Session id,1234
1,Target,Review
2,Target type,Binary
3,Target mapping,"Negative: 0, Positive: 1"
4,Original data shape,"(40250, 505)"
5,Transformed data shape,"(40250, 505)"
6,Transformed train set shape,"(28175, 505)"
7,Transformed test set shape,"(12075, 505)"
8,Numeric features,504
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:00:00
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
or_xgb_report2 = pull()

In [24]:
# UNDERSAMPLING

rs = rrs(random_state=1234)

rs_train_x2, rs_train_y2 = rs.fit_resample(X =train2.drop('Review', axis=1), y = train2['Review'])

rs_train2 = rs_train_x2.copy()
rs_train2['Review'] = rs_train_y2

under_set2 = setup(rs_train2, fold=5, target='Review', session_id=1234)
ur_xgb2 = under_set2.create_model('xgboost', fold=5, return_train_score=True, random_state=1234)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:01:47
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
ur_xgb_report2 = pull()